<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/PIPELINE_v0.5_ESIK%C3%84SITTELY%2C_EDA_PIIRTEET_BASELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
==========================================================================================
 TÄYDELLINEN PIPELINE v0.5 - HELSINGIN DATAN ESIKÄSITTELY, EDA, PIIRTEET & BASELINE
==========================================================================================

 Versio: 0.5
 Päivitetty: 2025-04-12 00:07 (EEST)
 Edellinen versio: 0.4 (Korjattu WindDirection-logiikka + Datan Jako & Baseline Malli)

 Skriptin tarkoitus:
 --------------------
 Tämä skripti muodostaa täydellisen data pipelinen Helsingin ilmanlaadun ja sään
 datalle (FMI), joka kattaa seuraavat vaiheet:
 1. Raakadatan lataus määritellyistä URL-osoitteista.
 2. Datan esikäsittely ja yhdistäminen (aikaleimat, nimeäminen, merge,
    duplikaatit, reindex, NaN-täyttö).
 3. Eksploratiivinen data-analyysi (EDA) (päällekkäisten sarakkeiden käsittely,
    negatiivisten korjaus, visualisoinnit, korrelaatiot, ajallinen analyysi).
 4. Piirteiden muokkaus (Feature Engineering) aikaan perustuvilla piirteillä.
 5. Datan kronologinen jako opetus- ja testijoukkoihin.
 6. Baseline-mallin (Lineaarinen Regressio) koulutus käyttäen *vain*
    aikaan perustuvia piirteitä.
 7. Baseline-mallin evaluointi testijoukolla (MAE, RMSE, R²).
 8. Lopullisen, kaikki piirteet sisältävän DataFramen tallentaminen
    Parquet-tiedostoon myöhempää käyttöä varten.

 Vaaditut kirjastot:
 -------------------
 - pandas, numpy, requests, io, os, matplotlib, seaborn
 - scikit-learn (LinearRegression, metrics)
 - pyarrow (Parquet-tallennukseen/lukuun)

 Käyttö:
 -------
 1. Varmista, että vaaditut kirjastot on asennettu (`pip install pandas numpy requests matplotlib seaborn scikit-learn pyarrow`).
 2. Kopioi tämä KOKO koodi huolellisesti editoriisi varmistaen, että sisennykset säilyvät oikein (erityisesti if/else-lohkoissa).
 3. Aja skripti. Se suorittaa koko pipelinen latauksesta mallin evaluointiin.
    Tulosteet ja kuvaajat (EDA-vaiheesta) näytetään suorituksen aikana.
 4. Lopuksi skripti tallentaa käsitellyn datan Parquet-tiedostoon
    `data/processed/Helsinki_Data_With_Features_Pipeline_v0.5.parquet`.
 5. Tämän jälkeen voit seuraavissa skripteissä/notebookeissa ladata suoraan
    tämän Parquet-tiedoston ja keskittyä esim. kehittyneempien mallien
    kouluttamiseen tai tarkempaan analyysiin.
"""

# === Vaihe 1: Alustus ja Kirjastojen Tuonti ===
print("="*60)
print(" Vaihe 1: Alustus ja Kirjastojen Tuonti")
print("="*60)

import pandas as pd
import numpy as np
import requests
import io
import os
import matplotlib.pyplot as plt
import seaborn as sns
# Tuodaan tarvittavat osat Scikit-learnista mallinnukseen ja evaluointiin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Asetetaan oletustyyli kuvaajille
plt.style.use('ggplot')

# --- Konfiguraatio ---
# Määritellään datatiedostojen URL-osoitteet sanakirjana
DATA_URLS = {
    "weather1": "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2011.4.2023%20-%2011.4.2025_4c0a0316-74e0-4792-9854-fd6315fbc965.csv",
    "weather2": "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2011.4.2023%20-%2011.4.2025_84370efb-e7a0-48ed-b991-0432c84c1475.csv",
    "aq": "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%2011.4.2023%20-%2011.4.2025_8bbe3500-d31d-459d-ac37-16b4bd64a2cc.csv"
}
# Määritellään tallennuspolku ja -nimi lopulliselle datalle piirteineen
PROCESSED_DATA_FOLDER = "data/processed"
PROCESSED_DATA_FILENAME_FE = "Helsinki_Data_With_Features_Pipeline_v0.5.parquet" # Päivitetty versio
PROCESSED_DATA_PATH_FE = os.path.join(PROCESSED_DATA_FOLDER, PROCESSED_DATA_FILENAME_FE)
TARGET_VARIABLE = 'Ozone' # Määritellään ennustettava kohdemuuttuja
TEST_DATA_RATIO = 0.2     # Testidatan osuus (esim. 20% lopusta)
SAVE_FINAL_DATA = True    # Lippu: tallennetaanko lopullinen data?


# === Apufunktiot (Lataus & Esikäsittely) ===

def _load_single_csv_from_url(url, key_name):
    """Apufunktio: Lataa YHDEN CSV-tiedoston URL-osoitteesta."""
    print(f"  Yritetään ladata {key_name}: {url[:70]}...")
    try:
        response = requests.get(url); response.raise_for_status()
        df = pd.read_csv(io.StringIO(response.text), sep=','); print(f"    -> {key_name} luettu.")
        return df
    except Exception as e: print(f"    -> VIRHE ({key_name}): {e}"); return None

def _preprocess_single_fmi_df(df, data_key_name):
    """Apufunktio: Esikäsittelee YHDEN FMI DataFrame:n (aika, numerot)."""
    if df is None: return None
    original_cols = df.columns.tolist(); date_cols = ['Vuosi', 'Kuukausi', 'Päivä']; time_col = 'Aika [Paikallinen aika]'
    if all(col in original_cols for col in date_cols) and time_col in original_cols:
        try:
            dt_str = df['Vuosi'].astype(str)+'-'+df['Kuukausi'].astype(str).str.zfill(2)+'-'+df['Päivä'].astype(str).str.zfill(2)+' '+df[time_col].astype(str)
            df['Timestamp'] = pd.to_datetime(dt_str); cols_to_drop = date_cols + [time_col]
            if 'Havaintoasema' in original_cols: cols_to_drop.append('Havaintoasema')
            df = df.drop(columns=cols_to_drop, errors='ignore').set_index('Timestamp')
        except Exception as e: print(f"    -> VIRHE aikaleiman muodostamisessa ({data_key_name}): {e}."); return None
    else: print(f"    -> VIRHE: Aikaleimasarakkeita ei löytynyt ({data_key_name})."); return None
    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]): df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(axis=1, how='all')
    return df

# === Pipeline Vaihe 1: Datan Lataus ===
def lataa_raakadata(url_dict):
    """Lataa raakadatatiedostot määritellyistä URL-osoitteista."""
    print("\n" + "="*60); print(" Pipeline Vaihe 1: Datan Lataus"); print("="*60)
    raw_dataframes = {}; success = True
    for key, url in url_dict.items():
        df_raw = _load_single_csv_from_url(url, key)
        if df_raw is None: success = False
        raw_dataframes[key] = df_raw
    if not success: print("\nVAROITUS: Kaikkien datatiedostojen lataus ei onnistunut."); return None
    print("\nKaikki datatiedostot ladattu onnistuneesti sanakirjaan.")
    return raw_dataframes

# === Pipeline Vaihe 2: Datan Esikäsittely ja Yhdistäminen ===
def esikasittele_yhdistetty_data(raw_dfs_dict):
    """Yhdistää ja esikäsittelee ladatut raakadatan DataFramet."""
    print("\n" + "="*60); print(" Pipeline Vaihe 2: Datan Esikäsittely ja Yhdistäminen"); print("="*60)
    required_keys = ['weather1', 'weather2', 'aq']
    if not isinstance(raw_dfs_dict, dict) or not all(key in raw_dfs_dict for key in required_keys): print("VIRHE: Syöte 'raw_dfs_dict' ei ole kelvollinen."); return None
    # --- 1. Esikäsittele ---
    df_w1 = _preprocess_single_fmi_df(raw_dfs_dict.get('weather1'), 'weather1')
    df_w2 = _preprocess_single_fmi_df(raw_dfs_dict.get('weather2'), 'weather2')
    df_aq = _preprocess_single_fmi_df(raw_dfs_dict.get('aq'), 'aq')
    if df_w1 is None or df_w2 is None or df_aq is None: print("VIRHE: Alustava esikäsittely epäonnistui."); return None
    # --- 2. Nimeä uudelleen ---
    print("\nUudelleennimetään sarakkeita...")
    r_map_w1 = {'Ilman lämpötila keskiarvo [°C]': 'Temperature_W1','Tuulen suunta keskiarvo [°]': 'WindDirection_W1', 'Keskituulen nopeus keskiarvo [m/s]': 'WindSpeed_W1','Näkyvyys keskiarvo [m]': 'Visibility','Pilvisyys [1/8]': 'Cloudiness', 'Ilmanpaine merenpinnan tasolla keskiarvo [hPa]': 'Pressure_SeaLevel'}
    v_map_w1 = {k: v for k, v in r_map_w1.items() if k in df_w1.columns}; df_w1 = df_w1.rename(columns=v_map_w1); print(f"  -> weather1: {df_w1.columns.tolist()}")
    r_map_w2 = {'Lämpötilan keskiarvo [°C]': 'Temperature_W2', 'Ylin lämpötila [°C]': 'Temperature_Max','Alin lämpötila [°C]': 'Temperature_Min', 'Keskituulen nopeus [m/s]': 'WindSpeed_W2','Tuulen suunnan keskiarvo [°]': 'WindDirection_W2','Ilmanpaineen keskiarvo [hPa]': 'Pressure_W2'}
    v_map_w2 = {k: v for k, v in r_map_w2.items() if k in df_w2.columns}; df_w2 = df_w2.rename(columns=v_map_w2); print(f"  -> weather2: {df_w2.columns.tolist()}")
    aq_map = {'Otsoni [µg/m3]': 'Ozone', 'Hengitettävät hiukkaset <10 µm [µg/m3]': 'PM10','Pienhiukkaset <2.5 µm [µg/m3]': 'PM25','Typpidioksidi [µg/m3]': 'NO2', 'Typpimonoksidi [µg/m3]': 'NO', 'Hiilimonoksidi [µg/m3]': 'CO', 'Rikkidioksidi [µg/m3]': 'SO2', 'Musta hiili [µg/m3]': 'BC'}
    aq_cols = [k for k in aq_map.keys() if k in df_aq.columns]
    if 'Otsoni [µg/m3]' not in aq_cols: print("-> VIRHE: Otsonisaraketta ei löytynyt!"); return None
    v_map_aq = {k: v for k, v in aq_map.items() if k in aq_cols}; df_aq = df_aq[aq_cols].rename(columns=v_map_aq); print(f"  -> aq: {df_aq.columns.tolist()}")
    # --- 3. Yhdistä ---
    print("\nYhdistetään DataFrameja..."); df_w = pd.merge(df_w1, df_w2, left_index=True, right_index=True, how='outer')
    df = pd.merge(df_w, df_aq, left_index=True, right_index=True, how='outer'); print(f"  -> Yhdistetty. Muoto: {df.shape}")
    # --- 4. Siivoa ---
    print("\nLopullinen siivous..."); df = df.sort_index()
    if not df.index.is_unique: print(f"-> Käsitellään {df.index.duplicated().sum()} duplikaattia..."); df = df.groupby(level=0).mean(); print("-> Duplikaatit käsitelty.")
    else: print("-> Ei duplikaatteja.")
    if not df.empty: print("-> Varmistetaan tuntifrekvenssi..."); min_ts, max_ts = df.index.min(), df.index.max(); df = df.reindex(pd.date_range(start=min_ts, end=max_ts, freq='h')); print(f"-> Uusi muoto: {df.shape}")
    else: return None
    print("-> Täytetään NaN..."); nan_b = df.isnull().sum().sum(); df = df.ffill().bfill()
    if df.isnull().sum().sum()==0: print(f"-> NaN täytetty (oli {nan_b}).")
    else: print(f"-> VAROITUS: {df.isnull().sum().sum()} NaN jäi!"); return None
    print("\nEsikäsittely valmis."); return df

# === Pipeline Vaihe 3: Datan Tutkiminen (EDA) ===
def tutki_dataa(df):
    """Suorittaa eksploratiivista data-analyysiä (EDA) esikäsitellylle DataFramelle."""
    print("\n" + "="*60); print(" Pipeline Vaihe 3: Datan Tutkiminen (EDA)"); print("="*60)
    if df is None or df.empty: print("VIRHE: Ei dataa tutkittavaksi."); return None
    df_eda = df.copy()

    # --- 3a: Päällekkäisten Sääsarakkeiden Käsittely (KORJATTU LOGIIKKA & SISENNYS) ---
    print("\n--- 3a: Päällekkäisten Sääsarakkeiden Käsittely ---")
    # Määritellään parit ja väliaikainen nimi keskiarvolle, jotta alkuperäiset voidaan poistaa
    duplicate_pairs = {
        'Temperature': ('Temperature_W1', 'Temperature_W2'),
        'WindSpeed': ('WindSpeed_W1', 'WindSpeed_W2'),
        'WindDirection_Avg': ('WindDirection_W1', 'WindDirection_W2'), # Käytetään väliaikaista nimeä
        'Pressure': ('Pressure_SeaLevel', 'Pressure_W2')
    }
    columns_to_drop = [] # Kerätään VAIN alkuperäiset poistettavat nimet

    for new_col_name, (col1, col2) in duplicate_pairs.items():
        # Tarkistetaan, että molemmat alkuperäiset sarakkeet ovat olemassa
        # 'IF' LOHKO ALKAA TÄSTÄ SISENNYKSESTÄ
        if col1 in df_eda.columns and col2 in df_eda.columns:
            # Koodi IF-lohkon sisällä sisennettynä
            print(f"  Käsitellään pari: {col1} & {col2} -> {new_col_name}")
            # Erityiskäsittely tuulen suunnalle (asteet 0-360)
            if new_col_name == 'WindDirection_Avg':
                 # Lasketaan vektorikeskiarvo tuulen suunnalle
                 print("    -> Käytetään vektorikeskiarvoa tuulen suunnalle.")
                 rad1 = np.deg2rad(df_eda[col1])
                 rad2 = np.deg2rad(df_eda[col2])
                 x_avg = (np.cos(rad1) + np.cos(rad2)) / 2.0
                 y_avg = (np.sin(rad1) + np.sin(rad2)) / 2.0
                 avg_rad = np.arctan2(y_avg, x_avg)
                 avg_deg = np.rad2deg(avg_rad)
                 # Muunnetaan välille [0, 360)
                 df_eda[new_col_name] = (avg_deg + 360) % 360
            else:
                 # Muille muuttujille normaali rivittäinen keskiarvo
                 df_eda[new_col_name] = df_eda[[col1, col2]].mean(axis=1)

            # Lisätään VAIN ALKUPERÄISET sarakkeet poistettavien listalle
            columns_to_drop.extend([col1, col2])
        # 'ELSE' LOHKO ALKAA TÄSTÄ SISENNYKSESTÄ (SAMALTA TASOLTA KUIN 'IF')
        # VARMISTA ETTEI TÄSSÄ OLE YLIMÄÄRÄISIÄ VÄLILYÖNTEJÄ ALUSSA
        else:
            # Koodi ELSE-lohkon sisällä sisennettynä yhden tason syvemmälle
            missing = [c for c in [col1, col2] if c not in df_eda.columns]
            print(f"  -> HUOM: Paria {c1}/{c2} ei voitu käsitellä. Puuttuu: {missing}")
            # Tässä ei tarvitse tehdä muuta

    # For-loopin jälkeinen koodi jatkuu tältä sisennystasolta
    columns_to_drop_unique = list(set(columns_to_drop))
    columns_to_drop_final = [col for col in columns_to_drop_unique if col in df_eda.columns]

    if columns_to_drop_final:
        df_eda = df_eda.drop(columns=columns_to_drop_final)
        print(f"  -> Poistettu alkuperäiset: {columns_to_drop_final}")
    else:
        print("  -> Ei päällekkäisiä sarakkeita poistettavaksi.")

    # Nimetään tuulen suunnan keskiarvo lopulliseen muotoon (jos se luotiin)
    # TÄMÄ TEHDÄÄN VAIN ALKUPERÄISTEN PUDOTUKSEN JÄLKEEN
    if 'WindDirection_Avg' in df_eda.columns:
        df_eda = df_eda.rename(columns={'WindDirection_Avg': 'WindDirection'})
        print("  -> Sarake 'WindDirection_Avg' nimetty 'WindDirection'.")

    print("  -> Päällekkäisten sarakkeiden käsittely valmis. Nykyiset:", df_eda.columns.tolist())

    # --- 3b: Negatiivisten Pitoisuuksien Käsittely ---
    print("\n--- 3b: Negatiivisten Pitoisuuksien Tarkistus ja Korjaus ---")
    conc_cols = ['Ozone','PM10','PM25','NO2','NO','SO2','BC','CO'] # Käytä uusia lyhyitä nimiä
    cols_chk = [c for c in conc_cols if c in df_eda.columns]; neg_c = (df_eda[cols_chk] < 0).sum()
    if (neg_c > 0).any(): print("  Korjataan negatiiviset..."); print("  Määrät:\n", neg_c[neg_c > 0].to_string())
    for c in cols_chk:
        if neg_c.get(c, 0) > 0: df_eda[c]=df_eda[c].clip(lower=0)
    print("  -> Korjattu.") if (neg_c > 0).any() else print("  -> Ei negatiivisia.")

    # --- 3c, 3d, 3e, 3f (Analyysit ja visualisoinnit) ---
    print("\n--- 3c: Kohdemuuttujan ('Ozone') Analyysi ---")
    if TARGET_VARIABLE not in df_eda.columns: print(f"VIRHE: Kohdemuuttuja '{TARGET_VARIABLE}' puuttuu!"); return None
    print(f"  Tilastot '{TARGET_VARIABLE}':"); print(df_eda[TARGET_VARIABLE].describe().to_string())
    print(f"  Visualisoidaan '{TARGET_VARIABLE}' aikasarja..."); plt.figure(figsize=(18, 5)); df_eda[TARGET_VARIABLE].plot(alpha=0.8, title=f'{TARGET_VARIABLE} ajan funktiona'); plt.ylabel(f'{TARGET_VARIABLE} (µg/m³)'); plt.xlabel('Aika'); plt.grid(True); plt.tight_layout(); plt.show()
    print(f"  Visualisoidaan '{TARGET_VARIABLE}' jakauma..."); plt.figure(figsize=(10, 5)); sns.histplot(df_eda[TARGET_VARIABLE], kde=True, bins=50); plt.title(f'{TARGET_VARIABLE} jakauma'); plt.xlabel(f'{TARGET_VARIABLE} (µg/m³)'); plt.ylabel('Frekvenssi / Tiheys'); plt.grid(True, axis='y'); plt.tight_layout(); plt.show()

    print("\n--- 3d: Korrelaatioanalyysi ---"); w_cols = [TARGET_VARIABLE, 'Temperature', 'WindSpeed', 'WindDirection', 'Pressure', 'Visibility', 'Temperature_Max', 'Temperature_Min']; w_cols = [c for c in w_cols if c in df_eda.columns]; print(f"  Sarakkeet: {w_cols}")
    corr_m = df_eda[w_cols].corr(); plt.figure(figsize=(9, 7)); sns.heatmap(corr_m, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5); plt.title(f'Korrelaatiot: {TARGET_VARIABLE} vs. Sää'); plt.show()
    print(f"\n  {TARGET_VARIABLE} korrelaatiot (laskeva):"); print(corr_m[TARGET_VARIABLE].sort_values(ascending=False).drop(TARGET_VARIABLE).to_string())

    print("\n--- 3e: Hajontakaaviot ---"); scatter_f = ['Temperature', 'WindSpeed', 'Visibility', 'Pressure']; scatter_f = [c for c in scatter_f if c in df_eda.columns]; print(f"  Piirretään: {TARGET_VARIABLE} vs {scatter_f}")
    n_f=len(scatter_f); n_r=(n_f+1)//2; plt.figure(figsize=(14, 5*n_r))
    for i, f in enumerate(scatter_f): plt.subplot(n_r, 2, i+1); sns.scatterplot(data=df_eda, x=f, y=TARGET_VARIABLE, alpha=0.1, s=5); plt.title(f'{TARGET_VARIABLE} vs. {f}'); plt.xlabel(f); plt.ylabel(f'{TARGET_VARIABLE} (µg/m³)'); plt.grid(True)
    plt.tight_layout(); plt.show()

    print("\n--- 3f: Ajallinen Analyysi ---"); print("  Kuukausivaihtelu..."); m_avg = df_eda.groupby(df_eda.index.month)[[TARGET_VARIABLE, 'Temperature']].mean(); m_avg.index.name = 'Kuukausi'
    fig, ax1 = plt.subplots(figsize=(12, 5)); ax2 = ax1.twinx(); m_avg[TARGET_VARIABLE].plot(ax=ax1, c='r', marker='o', label=TARGET_VARIABLE); m_avg['Temperature'].plot(ax=ax2, c='b', marker='s', ls='--', label='Lämpötila')
    ax1.set(ylabel=f'Keskim. {TARGET_VARIABLE} (µg/m³)', xlabel='Kuukausi', title='Keskimääräinen kuukausivaihtelu', xticks=m_avg.index); ax1.tick_params(axis='y', labelcolor='r'); ax2.set_ylabel('Keskim. Lämpötila (°C)', color='b'); ax2.tick_params(axis='y', labelcolor='b'); ax1.grid(True); fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes); fig.tight_layout(); plt.show()
    print("  Tuntivaihtelu..."); h_avg = df_eda.groupby(df_eda.index.hour)[[TARGET_VARIABLE, 'Temperature']].mean(); h_avg.index.name = 'Tunti'
    fig, ax1 = plt.subplots(figsize=(12, 5)); ax2 = ax1.twinx(); h_avg[TARGET_VARIABLE].plot(ax=ax1, c='r', marker='o', label=TARGET_VARIABLE); h_avg['Temperature'].plot(ax=ax2, c='b', marker='s', ls='--', label='Lämpötila')
    ax1.set(ylabel=f'Keskim. {TARGET_VARIABLE} (µg/m³)', xlabel='Tunti', title='Keskimääräinen tuntivaihtelu', xticks=h_avg.index[::2]); ax1.tick_params(axis='y', labelcolor='r'); ax2.set_ylabel('Keskim. Lämpötila (°C)', color='b'); ax2.tick_params(axis='y', labelcolor='b'); ax1.grid(True); fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes); fig.tight_layout(); plt.show()

    print("\nEDA-vaihe suoritettu."); return df_eda

# === Pipeline Vaihe 4: Piirteiden Muokkaus (Feature Engineering) ===
def muokkaa_piirteet(df):
    """Lisää dataan aikaan perustuvia piirteitä mallinnusta varten."""
    print("\n" + "="*60); print(" Pipeline Vaihe 4: Piirteiden Muokkaus"); print("="*60)
    if df is None or df.empty or not isinstance(df.index, pd.DatetimeIndex): print("VIRHE: Piirteiden muokkaus vaatii kelvollisen DataFramen."); return None, None
    df_feat = df.copy(); print("Lisätään aikaan perustuvia piirteitä...")
    idx = df_feat.index
    df_feat['hour'] = idx.hour; df_feat['dayofweek'] = idx.dayofweek; df_feat['dayofyear'] = idx.dayofyear
    df_feat['month'] = idx.month; df_feat['year'] = idx.year; df_feat['weekofyear'] = idx.isocalendar().week.astype(int)
    print("  -> Lineaariset: hour, dayofweek, dayofyear, month, year, weekofyear")
    df_feat['hour_sin']=np.sin(2*np.pi*df_feat['hour']/24.0); df_feat['hour_cos']=np.cos(2*np.pi*df_feat['hour']/24.0)
    df_feat['dayofweek_sin']=np.sin(2*np.pi*df_feat['dayofweek']/7.0); df_feat['dayofweek_cos']=np.cos(2*np.pi*df_feat['dayofweek']/7.0)
    df_feat['month_sin']=np.sin(2*np.pi*df_feat['month']/12.0); df_feat['month_cos']=np.cos(2*np.pi*df_feat['month']/12.0)
    print("  -> Sykliset: hour_sin/cos, dayofweek_sin/cos, month_sin/cos")
    time_feature_names = ['hour','dayofweek','dayofyear','month','year','weekofyear','hour_sin','hour_cos','dayofweek_sin','dayofweek_cos','month_sin','month_cos']
    print(f"  -> Lisätyt piirteet: {time_feature_names}")
    print("\nPiirteiden muokkaus valmis."); return df_feat, time_feature_names # Palautetaan myös piirteiden nimet

# === Pipeline Vaihe 5: Datan Jako ===
def jaa_data(df, target_sarake, test_osuus):
    """Jakaa datan kronologisesti opetus- ja testijoukkoihin."""
    print("\n" + "="*60); print(" Pipeline Vaihe 5: Datan Jako"); print("="*60)
    if df is None or df.empty: print("VIRHE: Ei dataa jaettavaksi."); return None, None, None, None
    if target_sarake not in df.columns: print(f"VIRHE: Kohdemuuttujaa '{target_sarake}' ei löydy."); return None, None, None, None
    n = len(df); split_index = int(n * (1 - test_osuus))
    if split_index <= 0 or split_index >= n: print(f"VIRHE: Testikoko {test_osuus} tuottaa virheellisen jakoindeksin {split_index}."); return None,None,None,None
    df_train = df.iloc[:split_index]; df_test = df.iloc[split_index:]
    print(f"Data jaettu: Opetus {df_train.shape[0]} riviä ({df_train.index.min()} - {df_train.index.max()})")
    print(f"            Testi  {df_test.shape[0]} riviä ({df_test.index.min()} - {df_test.index.max()})")
    # Varmistetaan, että target_sarake on olemassa ennen pudotusta
    if target_sarake in df.columns:
        feature_columns = df.columns.drop(target_sarake)
    else:
        print(f"VIRHE: Kohdemuuttujaa '{target_sarake}' ei löytynyt piirteiden erottelua varten."); return None,None,None,None
    X_train = df_train[feature_columns]; y_train = df_train[target_sarake]
    X_test = df_test[feature_columns]; y_test = df_test[target_sarake]
    print("Opetus- ja testijoukot luotu (X ja y)."); return X_train, y_train, X_test, y_test

# === Pipeline Vaihe 6: Baseline-mallin Koulutus ===
def kouluta_perusmalli(X_train, y_train, aikapiirteet):
    """Kouluttaa Lineaarisen Regression baseline-mallin käyttäen vain aikapiirteitä."""
    print("\n" + "="*60); print(" Pipeline Vaihe 6: Baseline-mallin Koulutus"); print("="*60)
    if X_train is None or y_train is None or not aikapiirteet: print("VIRHE: Virheelliset syötteet."); return None
    # Varmistetaan, että kaikki listatut aikapiirteet löytyvät X_trainista
    missing_features = [p for p in aikapiirteet if p not in X_train.columns]
    if missing_features: print(f"VIRHE: Seuraavia aikapiirteitä ei löydy X_trainista: {missing_features}."); return None
    print(f"Koulutetaan Lineaarinen Regressio käyttäen piirteitä: {aikapiirteet}")
    X_train_time_features = X_train[aikapiirteet]; model = LinearRegression()
    try: model.fit(X_train_time_features, y_train); print("Baseline-malli koulutettu."); return model
    except Exception as e: print(f"VIRHE mallin koulutuksessa: {e}"); return None

# === Pipeline Vaihe 7: Mallin Evaluointi ===
def evaluoi_malli(model, X, y, datasetti_nimi, aikapiirteet=None):
    """Evaluoi koulutetun mallin suorituskyvyn annetulla datajoukolla."""
    print("\n" + "="*60); print(f" Pipeline Vaihe 7: Mallin Evaluointi ({datasetti_nimi})"); print("="*60)
    if model is None or X is None or y is None: print("VIRHE: Virheelliset syötteet."); return
    X_eval = X; features_used = X.columns.tolist()
    if aikapiirteet:
        # Varmistetaan, että kaikki tarvittavat piirteet löytyvät X:stä
        missing_features = [p for p in aikapiirteet if p not in X.columns]
        if missing_features: print(f"VIRHE: Seuraavia evaluointiin tarvittavia aikapiirteitä ei löydy X-datasta ({datasetti_nimi}): {missing_features}."); return
        X_eval = X[aikapiirteet]; features_used = aikapiirteet
        print(f"Käytetään ennustamiseen vain piirteitä: {features_used}")
    try:
        y_pred = model.predict(X_eval); mae = mean_absolute_error(y, y_pred); rmse = np.sqrt(mean_squared_error(y, y_pred)); r2 = r2_score(y, y_pred)
        print(f"Suorituskykymetriikat ({datasetti_nimi}):"); print(f"  MAE:  {mae:.3f}"); print(f"  RMSE: {rmse:.3f}"); print(f"  R²:   {r2:.3f}")
        plt.figure(figsize=(18, 5)); plt.plot(y.index, y, label='Todelliset', alpha=0.8); plt.plot(y.index, y_pred, label='Ennusteet (Baseline)', alpha=0.8, linestyle='--')
        plt.title(f'Baseline Mallin Ennusteet vs. Todelliset ({datasetti_nimi})'); plt.xlabel('Aika'); plt.ylabel(TARGET_VARIABLE); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()
    except Exception as e: print(f"VIRHE mallin evaluoinnissa ({datasetti_nimi}): {e}")

# === Pipeline Vaihe 8: Tallenna Lopullinen Data (Valinnainen) ===
def tallenna_lopullinen_data(df, polku):
    """Tallentaa annetun DataFramen Parquet-tiedostoon."""
    print("\n" + "="*60); print(" Pipeline Vaihe 8: Lopullisen Datan Tallennus (Parquet)"); print("="*60)
    if df is None or df.empty: print("VIRHE: Ei dataa tallennettavaksi."); return False
    try: import pyarrow; print("'pyarrow' löytyy.")
    except ImportError: print("VAROITUS: 'pyarrow' puuttuu."); return False
    try:
        output_folder = os.path.dirname(polku);
        if output_folder: os.makedirs(output_folder, exist_ok=True)
        df.to_parquet(polku, index=True)
        print(f"\nLopullinen data piirteineen tallennettu: {polku}"); return True
    except Exception as e: print(f"\nVIRHE tallennuksessa {polku}: {e}"); return False

# === Pääsuorituslohko ===
if __name__ == "__main__":
    print("\n" + "="*80); print(" ALOITETAAN KOKO DATA PIPELINE v0.5"); print("="*80)
    # Alustetaan muuttujat Noneksi varmuuden vuoksi
    df_kasitelty, df_eda_valmis, df_piirteilla = None, None, None
    X_train, y_train, X_test, y_test = None, None, None, None
    aikapiirteiden_nimet, baseline_model = None, None
    tallennettu_ok = False

    # Ajetaan vaiheet peräkkäin try-except-lohkossa
    try:
        # Vaihe 1
        raaka_datat_dict = lataa_raakadata(DATA_URLS)
        if raaka_datat_dict is None: raise ValueError("Raakadatan lataus epäonnistui.")

        # Vaihe 2
        df_kasitelty = esikasittele_yhdistetty_data(raaka_datat_dict)
        if df_kasitelty is None: raise ValueError("Datan esikäsittely epäonnistui.")

        # Vaihe 3
        df_eda_valmis = tutki_dataa(df_kasitelty)
        if df_eda_valmis is None: raise ValueError("EDA-vaihe epäonnistui tai palautti None.")

        # Vaihe 4
        # Muokkaa_piirteet palauttaa nyt tuple: (DataFrame, lista_piirteistä)
        result_fe = muokkaa_piirteet(df_eda_valmis)
        if result_fe is not None and len(result_fe) == 2:
            df_piirteilla, aikapiirteiden_nimet = result_fe
        else:
            raise ValueError("Piirteiden muokkaus epäonnistui tai ei palauttanut odotettua tulosta.")

        # Vaihe 5
        # Varmistetaan, että df_piirteilla ja aikapiirteiden_nimet ovat olemassa
        if df_piirteilla is not None and aikapiirteiden_nimet is not None:
            X_train, y_train, X_test, y_test = jaa_data(df_piirteilla, TARGET_VARIABLE, test_osuus=TEST_DATA_RATIO)
        else: # Tämä haara ei pitäisi olla mahdollinen, jos edellinen vaihe onnistui
             raise ValueError("Tarvittavat muuttujat datan jakoon puuttuvat.")

        # Vaihe 6
        if X_train is not None and y_train is not None and aikapiirteiden_nimet is not None:
            baseline_model = kouluta_perusmalli(X_train, y_train, aikapiirteiden_nimet)
        else: raise ValueError("Datan jako epäonnistui, ei voida kouluttaa.")

        # Vaihe 7
        if baseline_model is not None and X_test is not None and y_test is not None and aikapiirteiden_nimet is not None:
             evaluoi_malli(baseline_model, X_test, y_test, "Testisetti", aikapiirteet=aikapiirteiden_nimet)
        else: raise ValueError("Baseline-mallin koulutus epäonnistui, ei voida evaluoida.")

        # Vaihe 8 (Tallennus)
        if SAVE_FINAL_DATA and df_piirteilla is not None:
            print("\n" + "-"*40); print(" LOPULLISEN DATAFRAMEN YHTEENVETO (SIS. PIIRTEET) "); print("-"*40)
            print("\nInfo:"); df_piirteilla.info()
            print("\n5 viimeistä riviä:"); print(df_piirteilla.tail())
            tallennettu_ok = tallenna_lopullinen_data(df_piirteilla, PROCESSED_DATA_PATH_FE)
            if tallennettu_ok: print(" -> Tallennus onnistui.")
            else: print(" -> Tallennus ei onnistunut tai sitä ei suoritettu.")
        elif not SAVE_FINAL_DATA:
            print("\nLopullisen datan tallennus ohitettu (SAVE_FINAL_DATA=False).")

    except Exception as e:
        # Napataan mahdolliset odottamattomat virheet pipelinen aikana
        print("\n" + "!"*80)
        print(f" KRIITTINEN VIRHE PIPELINEN SUORITUKSESSA: {e}")
        import traceback
        traceback.print_exc() # Tulostaa tarkemman virhejäljen
        print("!"*80)

    finally:
        # Tämä lohko suoritetaan aina, riippumatta siitä tapahtuiko virhe vai ei
        print("\n" + "="*80); print(" KOKO PIPELINE SUORITETTU LOPPUUN"); print("="*80)